In [1]:
tps = []
N = []
PE = []
timesteps = []
Ns = [100,200,300,400,500]
densities = [0.01, 0.005, 0.001, 0.001, 0.0005]
dts = []

In [2]:
import flowermd
import gsd
import gsd.hoomd
import hoomd
import matplotlib.pyplot as plt
import mbuild as mb
import numpy as np
import warnings
from cmeutils.visualize import FresnelGSD
from flowermd.base import Pack, Simulation, System, Molecule
from flowermd.base.forcefield import BaseHOOMDForcefield
from flowermd.library import KremerGrestBeadSpring, LJChain
from flowermd.library.forcefields import BeadSpring
from flowermd.utils import get_target_box_number_density
from mbuild.compound import Compound
from mbuild.lattice import Lattice
warnings.filterwarnings('ignore')
cpu = hoomd.device.CPU()
class Graphene(System):
    def __init__(
        self,
        x_repeat,
        y_repeat,
        n_layers,
        base_units=dict(),
        periodicity=(True, True, False),
    ):
        surface = mb.Compound(periodicity=periodicity)
        a = 3**.5
        lattice = Lattice(
            lattice_spacing=[a,a,a],
            lattice_vectors=  [[a,0,0],[a/2,3/2,0],[0,0,1]],
            lattice_points={"A": [[1/3,1/3,0], [2/3, 2/3, 0]]},
        ) 
        Flakium = Compound(name="F", element="F") # defines a carbon atom that will be used to populate lattice points
        layers = lattice.populate(
            compound_dict={"A": Flakium}, x=x_repeat, y=y_repeat, z=n_layers
        ) # populates the lattice using the previously defined carbon atom for every "A" site, repeated in all x,y, and z directions
        surface.add(layers) # adds populated carbon lattice layers to the 'surface' compound, which represents our graphene structure 
        surface.freud_generate_bonds("F", "F", dmin=0.9, dmax=1.1) # generates bonds depending on input distance range, scales with lattice
        surface_mol = Molecule(num_mols=1, compound=surface) # wraps into a Molecule object, creating "1" instance of this molecule

        super(Graphene, self).__init__(
            molecules=[surface_mol],
            base_units=base_units,
        )

    def _build_system(self):
        return self.all_molecules[0]
#OK, so we want to initialize a system with some chains and some flakes
N_chains = 100
initial_dens = 0.01
final_dens = 0.3
kg_chain = LJChain(lengths=10,num_mols=N_chains)
sheet = Graphene(x_repeat=5, y_repeat=5, n_layers=1, periodicity=(False, False, False))
system = Pack(molecules=[Molecule(compound=sheet.all_molecules[0], num_mols=10), kg_chain], 
              density=initial_dens, packing_expand_factor = 6, seed=2, overlap=1.1)
system.visualize()

/Users/joselyngutierrez/miniconda3/envs/flowermd-dev/lib/python3.12/site-packages/gmso/core/element.py:10: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [3]:
import unyt as u
target_box = get_target_box_number_density(density=final_dens*u.Unit("nm**-3"),n_beads=(500+(N_chains*10)))

In [4]:
# this FF is for research question
# WCA = 2.0 **1/6, kT = 3.0, needs thousand chains, maybe 10-12 flakes. large system, need lots of steps. 5e6 probably good. 
ff = BeadSpring(
    r_cut=2**1/6,  
    beads={
        "A": dict(epsilon=1.0, sigma=1.0),  # chains
        "F": dict(epsilon=1.0, sigma=1.0),  # flakes
    },
    bonds={
        "F-F": dict(r0=1.0, k=1000),
        "A-A": dict(r0=1.0, k=1000.0),  # increased k to avoid chain collapse
    },
    angles={
        "A-A-A": dict(t0=2* np.pi / 3., k=100.0),   # moderate stiffness for chains
        "F-F-F": dict(t0=2 * np.pi / 3., k=5000),
    },
    dihedrals={
        "A-A-A-A": dict(phi0=0.0, k=0, d=-1, n=2), #need to turn this on later, messed up with straight chains
        "F-F-F-F": dict(phi0=0.0, k=500, d=-1, n=2),
    }
)
dt = 0.0005
gsd = f"{N_chains}_10mer10f_{dt}dt.gsd"
log = f"{N_chains}_10mer10f_{dt}dt.txt"
sim = Simulation(initial_state=system.hoomd_snapshot, forcefield=ff.hoomd_forces, device=cpu, dt = dt, gsd_write_freq=int(1000), log_file_name = log, gsd_file_name = gsd)
#sim.run_update_volume(final_box_lengths=[(1500/0.8)**1/3, (1500/0.8)**1/3, (1500/0.8)**1/3], kT=6.0, n_steps=5e6,tau_kt=100*sim.dt,period=10,thermalize_particles=True)
sim.run_update_volume(final_box_lengths=target_box, kT=6.0, n_steps=5e6,tau_kt=100*sim.dt,period=10,thermalize_particles=True)
sim.run_NVT(n_steps=1e6, kT=7, tau_kt=dt*100)
sim.flush_writers()
sim_visualizer = FresnelGSD(gsd_file=gsd, frame=-1, view_axis=(1, 1, 1))
sim_visualizer.view()

*Warning*: dihedral.harmonic: specified K <= 0


Initializing simulation state from a gsd.hoomd.Frame.
Step 1000 of 5000000; TPS: 1308.86; ETA: 1.0 hours, 4.0 minutes
Step 2000 of 5000000; TPS: 1850.16; ETA: 45.0 minutes
Step 3000 of 5000000; TPS: 2148.27; ETA: 38.8 minutes
Step 4000 of 5000000; TPS: 2334.02; ETA: 35.7 minutes
Step 5000 of 5000000; TPS: 2464.88; ETA: 33.8 minutes
Step 6000 of 5000000; TPS: 2559.11; ETA: 32.5 minutes
Step 7000 of 5000000; TPS: 2631.43; ETA: 31.6 minutes
Step 8000 of 5000000; TPS: 2687.19; ETA: 31.0 minutes
Step 9000 of 5000000; TPS: 2733.81; ETA: 30.4 minutes
Step 10000 of 5000000; TPS: 2770.36; ETA: 30.0 minutes
Step 11000 of 5000000; TPS: 2802.2; ETA: 29.7 minutes
Step 12000 of 5000000; TPS: 2828.12; ETA: 29.4 minutes
Step 13000 of 5000000; TPS: 2851.64; ETA: 29.1 minutes
Step 14000 of 5000000; TPS: 2872.05; ETA: 28.9 minutes
Step 15000 of 5000000; TPS: 2890.44; ETA: 28.7 minutes
Step 16000 of 5000000; TPS: 2906.21; ETA: 28.6 minutes
Step 17000 of 5000000; TPS: 2920.82; ETA: 28.4 minutes
Step 18000 

RuntimeError: Particle with unique tag 585 is no longer in the simulation box.

Cartesian coordinates: 
x: 12.1925 y: 149.673 z: -6.17036
Fractional coordinates: 
f.x: 0.913983 f.y: 5.582 f.z: 0.290492
Local box lo: (-14.7258, -14.7258, -14.7258)
          hi: (14.7258, 14.7258, 14.7258)


In [ ]:
data = np.genfromtxt(log, names=True)

In [ ]:
N.append(N_chains)

In [ ]:
tps.append(np.average(data["flowermdbasesimulationSimulationtps"]))

In [ ]:
PE.append(data["mdcomputeThermodynamicQuantitiespotential_energy"])

In [ ]:
timesteps.append(data["flowermdbasesimulationSimulationtimestep"])

In [ ]:
densities.append(final_dens)
dts.append(dt)

In [ ]:
np.savetxt('N.txt', N, delimiter=' ')
np.savetxt('tps.txt', tps, delimiter=' ')
np.savetxt('PE.txt', PE, delimiter=' ')
np.savetxt('timesteps.txt', timesteps, delimiter=' ')
np.savetxt('densities.txt', densities, delimiter=' ')
np.savetxt('dts.txt', dts, delimiter=' ')